In [2]:
import os
import tensorflow as tf 
import numpy as np

In [3]:
root_dir='../dataset/scan'
in_dir='../dataset/scan/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
num_img=len(img_paths)
from imageio import imread, imsave
photo = imread(img_paths[0])
photo.shape

(968, 1296, 3)

In [4]:
# 这些函数封装的真难用
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def write(input_file, output_file):
    root_dir = input_file
    img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
    num_img=len(img_paths)
    file_names = [(root_dir+'/color/{}.jpg'.format(f)).encode() for f in range(num_img)]
    writer = tf.python_io.TFRecordWriter(output_file) #定义writer，传入目标文件路径
    
    for i in range(num_img-15):
        rgb1_filename = ("{}.jpg".format(i)).encode()
        rgb2_filename = ('/color/'+"{}.jpg".format(i+10)).encode()
        depth1_filename = ('/depth/'+"{}.png".format(i)).encode()
        depth2_filename = ('/depth/'+"{}.png".format(i+10)).encode()
        shape1 = (968, 1296, 3)
        shape2 = (968, 1296, 3)
#             bbox1
#             bbox2
        c1Tw = np.loadtxt(root_dir+'/pose/'+"{}.txt".format(i),dtype=np.float32).ravel()
        c2Tw = np.loadtxt(root_dir+'/pose/'+"{}.txt".format(i+10),dtype=np.float32).ravel()
        K1 = np.loadtxt(root_dir+'/intrinsic/'+"intrinsic_color.txt",dtype=np.float32).ravel()
        K2 = np.loadtxt(root_dir+'/intrinsic/'+"intrinsic_color.txt",dtype=np.float32).ravel()
        tf_example = tf.train.Example(
            features=tf.train.Features(feature={
                'rgb1_filename': bytes_feature(rgb1_filename),
                'rgb2_filename': bytes_feature(rgb2_filename),
                'depth1_filename': bytes_feature(depth1_filename),
                'depth2_filename': bytes_feature(depth2_filename),
                'shape1': tf.train.Feature(int64_list=tf.train.Int64List(value=list(shape1))),
                'shape2': tf.train.Feature(int64_list=tf.train.Int64List(value=list(shape2))),
    #             'bbox1': int64_feature(), # x1,x2,y1,y2
    #             'bbox2': int64_feature(),
                'c1Tw': tf.train.Feature(float_list=tf.train.FloatList(value=c1Tw)),
                'c2Tw': tf.train.Feature(float_list=tf.train.FloatList(value=c2Tw)),
                'K1': tf.train.Feature(float_list=tf.train.FloatList(value=K1)),
                'K2': tf.train.Feature(float_list=tf.train.FloatList(value=K2)),
        }))

        #example序列化，并写入文件
        writer.write(tf_example.SerializeToString())
    writer.close()

In [5]:
write('../dataset/scan', 're.tfrecord')

In [5]:
def read_and_decode(file_name):
    filename_queue = tf.train.string_input_producer([file_name])
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                features={
                    'rgb1_filename': tf.FixedLenFeature([], tf.string),
                    'rgb2_filename': tf.FixedLenFeature([], tf.string),
                    'depth1_filename': tf.FixedLenFeature([], tf.string),
                    'depth2_filename': tf.FixedLenFeature([], tf.string),
                    'shape1': tf.FixedLenFeature([3], tf.int64),
                    'shape2': tf.FixedLenFeature([3], tf.int64),
    #                 'bbox1': tf.FixedLenFeature([4], tf.int64), # x1,x2,y1,y2
    #                 'bbox2': tf.FixedLenFeature([4], tf.int64),
                    'c1Tw': tf.FixedLenFeature([16], tf.float32),
                    'c2Tw': tf.FixedLenFeature([16], tf.float32),
                    'K1': tf.FixedLenFeature([16], tf.float32),
                    'K2': tf.FixedLenFeature([16], tf.float32),
                                                            })
    return features

In [6]:
features=read_and_decode('re.tfrecord')

In [9]:
with tf.Session() as sess: #开始一个会话
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    x= sess.run(features['rgb2_filename'])#在会话中取出image和label
    print(x)

b'../dataset/scan/color/10.jpg'
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Session has been closed.
